# Cài đặt thư viện

In [ ]:
!pip install ffmpeg-python tqdm boto3 requests pandas -q

In [ ]:
!pip install git+https://github.com/salaniz/pycocoevalcap.git -q

# Tải Bert Base model

In [ ]:
%cd UniVL-Video-captioning

In [ ]:
import sys
sys.path.append('./')

In [ ]:
%%bash
# Tạo thư mục cho BERT
mkdir -p modules/bert-base-uncased
cd modules/bert-base-uncased

# Download vocab file
wget -q https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
mv bert-base-uncased-vocab.txt vocab.txt

# Download BERT model
wget -q https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz
tar -xzf bert-base-uncased.tar.gz
rm bert-base-uncased.tar.gz

cd ../..
echo "✓ BERT model downloaded successfully!"
ls -lh modules/bert-base-uncased/

# Tải S3D HowTo100M pretrained weights

In [ ]:
%%bash
# Tạo thư mục cho weights (theo cấu trúc VideoFeatureExtractor)
mkdir -p VideoFeatureExtractor/model

# Download S3D HowTo100M weights (~500MB)
echo "Downloading S3D HowTo100M weights (this may take a few minutes)..."
wget -q --show-progress https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/s3d_howto100m.pth -O VideoFeatureExtractor/model/s3d_howto100m.pth

echo "✓ S3D weights downloaded to VideoFeatureExtractor/model/"
ls -lh VideoFeatureExtractor/model/

# Extract video feature từ S3D

In [ ]:
import sys
import os
import torch
import numpy as np
import math
import torch.nn.functional as F
from pathlib import Path

# Add VideoFeatureExtractor vào path (nếu chưa có)
if './VideoFeatureExtractor' not in sys.path:
    sys.path.insert(0, './VideoFeatureExtractor')

def extract_s3d_features_per_second(video_path, s3d_weights_path='VideoFeatureExtractor/model/s3d_howto100m.pth', 
                                     device='cuda'):
    """
    Extract S3D features với "1 feature per second" (MATCHING MSRVTT training dataset)
    
    ✅ CÁCH EXTRACT NÀY KHỚP VỚI MSRVTT TRAINING DATASET
    
    Logic:
    - Decode video với fps=16 (16 frames/second, S3D input requirement)
    - Sliding window stride=16 frames (1 second)
    - Video dài N seconds → N features [N, 1024]
    
    Args:
        video_path: Đường dẫn đến video file
        s3d_weights_path: Đường dẫn đến S3D weights
        device: 'cuda' hoặc 'cpu'
    
    Returns:
        features: numpy array shape [num_seconds, 1024]
    """
    # Import từ VideoFeatureExtractor
    from VideoFeatureExtractor.video_loader import VideoLoader
    from VideoFeatureExtractor.videocnn.models import s3dg
    import tempfile
    import pandas as pd
    import ffmpeg
    
    print(f"📹 Extracting S3D features from: {video_path}")
    print(f"   Method: 1 feature per second (matching MSRVTT training)")
    
    # 1. Load S3D model
    print("🔧 Loading S3D model...")
    model = s3dg.S3D(last_fc=False)
    model = model.to(device)
    model_data = torch.load(s3d_weights_path)
    
    # Load weights
    from VideoFeatureExtractor.model import init_weight
    model = init_weight(model, model_data)
    model.eval()
    print("✓ Model loaded")
    
    # 2. Get video info
    probe = ffmpeg.probe(video_path)
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    
    # Get duration in seconds
    if 'duration' in video_stream:
        duration = float(video_stream['duration'])
    else:
        duration = float(probe['format']['duration'])
    
    num_seconds = int(duration)
    print(f"🎬 Video duration: {duration:.2f}s → {num_seconds} seconds")
    
    # 3. Tạo temporary CSV cho VideoLoader
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv') as f:
        temp_csv = f.name
        df = pd.DataFrame({
            'video_path': [video_path],
            'feature_path': ['dummy.npy']
        })
        df.to_csv(temp_csv, index=False)
    
    try:
        # 4. Load video với fps=16 (16 frames/second, S3D requirement)
        print("🎬 Loading video with ffmpeg (fps=16)...")
        SIZE = 224
        CENTERCROP = True
        FPS = 16  # ✅ Decode 16 frames per second (S3D requires 16-frame clips)
        
        dataset = VideoLoader(
            temp_csv,
            framerate=FPS,  # 16 fps
            size=SIZE,
            centercrop=CENTERCROP
        )
        
        data = dataset[0]
        video = data['video']  # [T, C, H, W] where T = duration * fps
        
        if video.shape[0] <= 1:
            raise ValueError(f"Failed to decode video: {video_path}")
        
        num_decoded_frames = len(video)
        print(f"✓ Loaded {num_decoded_frames} frames (fps={FPS})")
        
        # 5. Extract features với sliding window stride=16 frames (1 second)
        print("🔮 Extracting features (1 feature per second)...")
        
        # Normalize to [0, 1]
        video = video / 255.0
        
        # Số clips = số giây (mỗi giây có 16 frames → 1 clip)
        num_clips = num_seconds
        features_list = []
        
        with torch.no_grad():
            for i in range(num_clips):
                start_frame = i * 16
                end_frame = start_frame + 16
                
                # Get 16 frames cho clip này
                if end_frame <= num_decoded_frames:
                    clip = video[start_frame:end_frame]  # [16, C, H, W]
                else:
                    # Pad nếu không đủ 16 frames
                    clip = video[start_frame:]
                    padding_needed = 16 - len(clip)
                    padding = torch.zeros(padding_needed, clip.shape[1], clip.shape[2], clip.shape[3])
                    clip = torch.cat([clip, padding], dim=0)
                
                # Reshape to [1, C, T, H, W] for S3D input
                clip = clip.permute(1, 0, 2, 3).unsqueeze(0)  # [1, 3, 16, 224, 224]
                clip = clip.to(device)
                
                # Extract feature
                feature = model(clip)  # [1, 1024]
                feature = F.normalize(feature, dim=1)  # L2 normalize
                features_list.append(feature.cpu())
        
        # Concat all features
        features = torch.cat(features_list, dim=0).numpy()  # [num_seconds, 1024]
        
        print(f"✓ Features extracted! Shape: {features.shape}")
        print(f"   {features.shape[0]} features (1 feature per second)")
        
        return features
        
    finally:
        # Cleanup
        if os.path.exists(temp_csv):
            os.unlink(temp_csv)


# Load model UniVL từ checkpoint

In [ ]:
import os
import sys
import torch
from modules.tokenization import BertTokenizer
from modules.modeling import UniVL
from modules.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

import argparse

# Tạo parser giống như trong main_task_caption.py
parser = argparse.ArgumentParser(description='UniVL Inference')

# Required arguments với default values (đủ để khởi tạo model)
parser.add_argument("--bert_model", default="bert-base-uncased", type=str)
parser.add_argument("--visual_model", default="visual-base", type=str)
parser.add_argument("--cross_model", default="cross-base", type=str)
parser.add_argument("--decoder_model", default="decoder-base", type=str)
parser.add_argument("--do_lower_case", action='store_true', default=True)
parser.add_argument("--cache_dir", default="", type=str)

# Model architecture params
parser.add_argument('--max_words', type=int, default=48)
parser.add_argument('--max_frames', type=int, default=48)
parser.add_argument('--video_dim', type=int, default=1024)
parser.add_argument('--text_num_hidden_layers', type=int, default=12)
parser.add_argument('--visual_num_hidden_layers', type=int, default=6)
parser.add_argument('--cross_num_hidden_layers', type=int, default=2)
parser.add_argument('--decoder_num_hidden_layers', type=int, default=3)

# Task config
parser.add_argument('--stage_two', action='store_true', default=True)
parser.add_argument('--task_type', type=str, default='caption')
parser.add_argument("--do_pretrain", action='store_true', default=False)  # Inference mode
parser.add_argument("--do_train", action='store_true', default=False)
parser.add_argument("--do_eval", action='store_true', default=False)

# Loss-related params (cần cho khởi tạo model)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--n_gpu', type=int, default=1)
parser.add_argument('--n_pair', type=int, default=1)
parser.add_argument('--margin', type=float, default=0.1)
parser.add_argument('--negative_weighting', type=int, default=1)
parser.add_argument('--hard_negative_rate', type=float, default=0.5)
parser.add_argument('--use_mil', action='store_true', default=False)
parser.add_argument('--sampled_use_mil', action='store_true', default=False)

# Distributed training
parser.add_argument('--local_rank', type=int, default=0)
parser.add_argument('--world_size', type=int, default=1)

# Parse với empty args (sử dụng tất cả default values)
args = parser.parse_args([])

print("✓ Arguments initialized with default values from training script")

# Load tokenizer
print("\n🔧 Loading tokenizer...")
tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)
print(f"✓ Tokenizer loaded (vocab size: {len(tokenizer.vocab)})")

# Load checkpoint
checkpoint_path = "/kaggle/input/best-univl/pytorch_model.bin"
print(f"\n🎯 Loading checkpoint from: {checkpoint_path}")

if os.path.exists(checkpoint_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"   Device: {device}")
    
    # Load model
    model_state_dict = torch.load(checkpoint_path, map_location='cpu')
    cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed')
    
    model = UniVL.from_pretrained(
        args.bert_model, args.visual_model, args.cross_model, args.decoder_model,
        cache_dir=cache_dir, state_dict=model_state_dict, task_config=args
    )
    
    model.to(device)
    model.eval()
    
    print("✓ Model loaded and ready for inference!")
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")
    print("\n📝 Using argparse with default values (same as training script)")
    print("   This ensures all required attributes are present!")
else:
    print(f"❌ Checkpoint not found: {checkpoint_path}")
    print("\n💡 TIP: Đảm bảo file checkpoint tồn tại và đường dẫn đúng.")


# Hàm inference

In [ ]:
import torch
import numpy as np
from modules.beam import Beam

# ✅ Helper functions for beam search (copied from main_task_caption.py to avoid distributed init issues)

def get_inst_idx_to_tensor_position_map(inst_idx_list):
    """Get the positional map of active instances in the batch."""
    return {inst_idx: tensor_position for tensor_position, inst_idx in enumerate(inst_idx_list)}

def collect_active_part(beamed_tensor, curr_active_inst_idx, n_prev_active_inst, n_bm):
    """Collect tensor parts associated with active instances."""
    _, *d_hs = beamed_tensor.size()
    n_curr_active_inst = len(curr_active_inst_idx)
    new_shape = (n_curr_active_inst * n_bm, *d_hs)

    beamed_tensor = beamed_tensor.view(n_prev_active_inst, -1)
    beamed_tensor = beamed_tensor.index_select(0, curr_active_inst_idx)
    beamed_tensor = beamed_tensor.view(*new_shape)

    return beamed_tensor

def collate_active_info(input_tuples, inst_idx_to_position_map, active_inst_idx_list, n_bm, device):
    """Collate active beam instances."""
    assert isinstance(input_tuples, tuple)
    sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt, video_mask_rpt = input_tuples

    n_prev_active_inst = len(inst_idx_to_position_map)
    active_inst_idx = [inst_idx_to_position_map[k] for k in active_inst_idx_list]
    active_inst_idx = torch.LongTensor(active_inst_idx).to(device)

    active_sequence_output_rpt = collect_active_part(sequence_output_rpt, active_inst_idx, n_prev_active_inst, n_bm)
    active_visual_output_rpt = collect_active_part(visual_output_rpt, active_inst_idx, n_prev_active_inst, n_bm)
    active_input_ids_rpt = collect_active_part(input_ids_rpt, active_inst_idx, n_prev_active_inst, n_bm)
    active_input_mask_rpt = collect_active_part(input_mask_rpt, active_inst_idx, n_prev_active_inst, n_bm)
    active_video_mask_rpt = collect_active_part(video_mask_rpt, active_inst_idx, n_prev_active_inst, n_bm)
    active_inst_idx_to_position_map = get_inst_idx_to_tensor_position_map(active_inst_idx_list)

    return (active_sequence_output_rpt, active_visual_output_rpt, active_input_ids_rpt, 
            active_input_mask_rpt, active_video_mask_rpt), active_inst_idx_to_position_map

def beam_decode_step(decoder, inst_dec_beams, len_dec_seq, inst_idx_to_position_map, n_bm, device, input_tuples):
    """Decode and update beam status."""
    assert isinstance(input_tuples, tuple)

    def prepare_beam_dec_seq(inst_dec_beams, len_dec_seq):
        dec_partial_seq = [b.get_current_state() for b in inst_dec_beams if not b.done]
        dec_partial_seq = torch.stack(dec_partial_seq).to(device)
        dec_partial_seq = dec_partial_seq.view(-1, len_dec_seq)
        return dec_partial_seq

    def predict_word(next_decoder_ids, n_active_inst, n_bm, device, input_tuples):
        sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt, video_mask_rpt = input_tuples
        next_decoder_mask = torch.ones(next_decoder_ids.size(), dtype=torch.uint8).to(device)

        dec_output = decoder(sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt,
                             video_mask_rpt, next_decoder_ids, next_decoder_mask, shaped=True, get_logits=True)
        dec_output = dec_output[:, -1, :]
        word_prob = torch.nn.functional.log_softmax(dec_output, dim=1)
        word_prob = word_prob.view(n_active_inst, n_bm, -1)
        return word_prob

    def collect_active_inst_idx_list(inst_beams, word_prob, inst_idx_to_position_map):
        active_inst_idx_list = []
        for inst_idx, inst_position in inst_idx_to_position_map.items():
            is_inst_complete = inst_beams[inst_idx].advance(word_prob[inst_position])
            if not is_inst_complete:
                active_inst_idx_list += [inst_idx]
        return active_inst_idx_list

    n_active_inst = len(inst_idx_to_position_map)
    dec_seq = prepare_beam_dec_seq(inst_dec_beams, len_dec_seq)
    word_prob = predict_word(dec_seq, n_active_inst, n_bm, device, input_tuples)

    active_inst_idx_list = collect_active_inst_idx_list(inst_dec_beams, word_prob, inst_idx_to_position_map)

    return active_inst_idx_list

def collect_hypothesis_and_scores(inst_dec_beams, n_best):
    """Collect final hypothesis and scores."""
    all_hyp, all_scores = [], []
    for inst_idx in range(len(inst_dec_beams)):
        scores, tail_idxs = inst_dec_beams[inst_idx].sort_scores()
        all_scores += [scores[:n_best]]
        hyps = [inst_dec_beams[inst_idx].get_hypothesis(i) for i in tail_idxs[:n_best]]
        all_hyp += [hyps]
    return all_hyp, all_scores

# Main inference function
def generate_caption_from_features(video_features, model, tokenizer, device='cuda', 
                                   max_frames=48, beam_size=5, max_words=48):
    """
    Generate caption từ video features
    
    ✅ Logic copied 100% từ eval_epoch() trong main_task_caption.py (dòng 560-640)
    
    Args:
        video_features: numpy array [num_frames, 1024]
        model: Trained UniVL model
        tokenizer: BERT tokenizer
        device: 'cuda' hoặc 'cpu'
        max_frames: Max frames (48)
        beam_size: Beam search size (5)
        max_words: Max caption length (48)
    
    Returns:
        caption: Generated caption string
    """
    model.eval()
    
    # Preprocess features
    num_frames = video_features.shape[0]
    
    # Truncate hoặc pad về max_frames=48
    if num_frames > max_frames:
        video_features = video_features[:max_frames]
        num_frames = max_frames
    else:
        padding = np.zeros((max_frames - num_frames, 1024), dtype=np.float32)
        video_features = np.concatenate([video_features, padding], axis=0)
    
    # Create mask
    video_mask = np.zeros(max_frames, dtype=np.int64)
    video_mask[:num_frames] = 1
    
    # Convert to tensors (batch size = 1)
    video = torch.from_numpy(video_features).unsqueeze(0).float().to(device)  # [1, 48, 1024]
    video_mask = torch.from_numpy(video_mask).unsqueeze(0).long().to(device)  # [1, 48]
    
    # ✅ Tạo dummy input theo đúng source (line 574-576)
    input_ids = torch.zeros(1, 1, dtype=torch.long).to(device)
    input_ids[0, 0] = tokenizer.vocab["[CLS]"]
    input_mask = torch.ones(1, 1, dtype=torch.long).to(device)
    segment_ids = torch.zeros(1, 1, dtype=torch.long).to(device)  # ✅ Cần segment_ids!
    
    with torch.no_grad():
        # ✅ Dùng get_sequence_visual_output() như source (line 579)
        sequence_output, visual_output = model.get_sequence_visual_output(
            input_ids, segment_ids, input_mask, video, video_mask
        )
        
        # ✅ Beam search logic (copied từ line 580-620)
        n_bm = beam_size
        device = sequence_output.device
        n_inst, len_s, d_h = sequence_output.size()
        _, len_v, v_h = visual_output.size()
        
        decoder = model.decoder_caption
        
        # Note: shaped first (line 587-589)
        input_ids = input_ids.view(-1, input_ids.shape[-1])
        input_mask = input_mask.view(-1, input_mask.shape[-1])
        video_mask = video_mask.view(-1, video_mask.shape[-1])
        
        # Repeat for beam (line 591-595)
        sequence_output_rpt = sequence_output.repeat(1, n_bm, 1).view(n_inst * n_bm, len_s, d_h)
        visual_output_rpt = visual_output.repeat(1, n_bm, 1).view(n_inst * n_bm, len_v, v_h)
        input_ids_rpt = input_ids.repeat(1, n_bm).view(n_inst * n_bm, len_s)
        input_mask_rpt = input_mask.repeat(1, n_bm).view(n_inst * n_bm, len_s)
        video_mask_rpt = video_mask.repeat(1, n_bm).view(n_inst * n_bm, len_v)
        
        # Prepare beams (line 597-601)
        inst_dec_beams = [Beam(n_bm, device=device, tokenizer=tokenizer) for _ in range(n_inst)]
        active_inst_idx_list = list(range(n_inst))
        inst_idx_to_position_map = get_inst_idx_to_tensor_position_map(active_inst_idx_list)
        
        # Decode (line 602-613)
        for len_dec_seq in range(1, max_words + 1):
            active_inst_idx_list = beam_decode_step(
                decoder, inst_dec_beams, len_dec_seq, inst_idx_to_position_map, n_bm, device,
                (sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt, video_mask_rpt)
            )
            
            if not active_inst_idx_list:
                break
            
            (sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt, video_mask_rpt), \
            inst_idx_to_position_map = collate_active_info(
                (sequence_output_rpt, visual_output_rpt, input_ids_rpt, input_mask_rpt, video_mask_rpt),
                inst_idx_to_position_map, active_inst_idx_list, n_bm, device
            )
        
        # Collect results (line 615-616)
        batch_hyp, batch_scores = collect_hypothesis_and_scores(inst_dec_beams, 1)
        result_list = [batch_hyp[i][0] for i in range(n_inst)]
        
        # Decode to text (line 620-634)
        for re_list in result_list:
            decode_text_list = tokenizer.convert_ids_to_tokens(re_list)
            if "[SEP]" in decode_text_list:
                SEP_index = decode_text_list.index("[SEP]")
                decode_text_list = decode_text_list[:SEP_index]
            if "[PAD]" in decode_text_list:
                PAD_index = decode_text_list.index("[PAD]")
                decode_text_list = decode_text_list[:PAD_index]
            decode_text = ' '.join(decode_text_list)
            decode_text = decode_text.replace(" ##", "").strip("##").strip()
            
            return decode_text


# Video to Caption

In [ ]:
def video_to_caption(video_path, model, tokenizer, 
                     s3d_weights_path='VideoFeatureExtractor/model/s3d_howto100m.pth',
                     beam_size=5, verbose=True):
    """
    End-to-End pipeline: Video → Caption
    
    Args:
        video_path: Path to video file
        model: Trained UniVL model
        tokenizer: BERT tokenizer
        s3d_weights_path: Path to S3D HowTo100M weights
        device: 'cuda' or 'cpu'
        beam_size: Beam search size (2-5)
        verbose: Print progress
    
    Returns:
        caption: Generated caption string
    """
    if verbose:
        print(f"🎬 Processing video: {video_path}")
        print("="*60)
    
    # Step 1: Extract S3D features
    if verbose:
        print("\n📹 Step 1/2: Extracting S3D features...")
    
    features = extract_s3d_features_per_second(
        video_path=video_path,
        s3d_weights_path='VideoFeatureExtractor/model/s3d_howto100m.pth',
        device='cuda' if torch.cuda.is_available() else 'cpu',
        batch_size=8
    )
    
    if verbose:
        print(f"   ✓ Features extracted: {features.shape}")
    
    # Step 2: Generate caption
    if verbose:
        print("\n💬 Step 2/2: Generating caption...")
    
    caption = generate_caption_from_features(
        video_features=features,
        model=model,
        tokenizer=tokenizer,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        max_frames=48,
        beam_size=beam_size,
        max_words=48
    )
    
    if verbose:
        print(f"   ✓ Caption generated!")
        print("\n" + "="*60)
        print(f"📝 Caption: {caption}")
        print("="*60)
    
    return caption

In [ ]:
video_path = "/kaggle/input/video-captioning-dataset/Video_Datasets/Video_Datasets/Test/video7010.mp4"

caption = video_to_caption(
    video_path=video_path,
    model=model,
    tokenizer=tokenizer,
    beam_size=5
)